In [1]:
!pip install transformers[torch] -U
!pip install accelerate -U


  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/d0/a7/7eedcf6a359e1e1eff3bc204ad022485aa5d88c08e1e3e0e0aee8a2e2235/transformers-4.47.0-py3-none-any.whl.metadata
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.4
    Uninstalling transformers-4.43.4:
      Successfully uninstalled transformers-4.43.4


In [2]:
import sys
print(sys.executable)
import accelerate
print(accelerate.__version__)

C:\Users\anany\anaconda3\python.exe
1.2.0


### The following code is for Train the BERT Transformer.

In [1]:
#Work perfect for Traing
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load the dataset
data_path = "C:\\Users\\anany\\OneDrive\\Desktop\\MS_Thesis\\Results of LLM\\Experiments for Runtime\\subset_runtime_dataset_transformer.csv"
df = pd.read_csv(data_path)
df = df[['SQL_Statement', 'Original']]  # Ensure you're focusing on the right columns
df['Original'] = df['Original'].apply(lambda x: 1 if x == 'YES' else 0)  # Convert labels to binary

# Convert DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)  # Split into training and testing

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#def preprocess_function(examples):
    #return tokenizer(examples['SQL_Statement'], truncation=True, padding="max_length", max_length=512)
#tokenized_datasets = dataset.map(preprocess_function, batched=True)


def preprocess_function(examples):
    # Tokenize the texts and include the labels
    result = tokenizer(examples['SQL_Statement'], truncation=True, padding="max_length", max_length=512)
    result['labels'] = examples['Original']  # Ensure 'Original' column has your labels and is integer-encoded
    return result

tokenized_datasets = dataset.map(preprocess_function, batched=True)




# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

# Evaluate the model
results = trainer.evaluate()
#print(results)


Map:   0%|          | 0/9043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1005 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\anany\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'eval_loss': 0.6034886240959167, 'eval_model_preparation_time': 0.0051, 'eval_runtime': 1094.3614, 'eval_samples_per_second': 0.918, 'eval_steps_per_second': 0.058}


### The following code is for Test the Transformer for a single input.

In [2]:
#work perfect for Testing just a single input
import numpy as np

# Assuming previous steps are already done, and the model is trained

# Preprocess new data (this function might already be defined in your script)
def preprocess_new_data(new_data):
    tokenized_data = tokenizer(new_data, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    return Dataset.from_dict({key: tokenized_data[key].detach().numpy() for key in tokenized_data})

# Example new data
new_data = ["select top 12000 p.objid, p.field, p.ra, p.dec, p.b, p.l, p.run, p.mjd, s.plate, s.fiberid, s.z, s.class from specobj as s join photoobj as p on p.objid=s.bestobjid where p.ra between 150 and 240 and p.dec between 0 and 3 and s.z between 0.005 and 0.2 and p.r between 13.5 and 17.7"]

# Process new data
new_dataset = preprocess_new_data(new_data)

# Get predictions
predictions = trainer.predict(new_dataset)

# Extract logits and convert them to class predictions
logits = predictions.predictions
predicted_classes = np.argmax(logits, axis=1)

# Convert numeric predictions to labels
predicted_labels = ['YES' if label == 1 else 'NO' for label in predicted_classes]

# Print the results
print("Predicted labels:", predicted_labels)


Predicted labels: ['NO']


### The following code is for Test the Transformer for a csv file input.

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report

# Load the test CSV file
data_path = "C:\\Users\\anany\\OneDrive\\Desktop\\MS_Thesis\\Results of LLM\\Experiments for Runtime\\sdss_runtime.csv"
df_test = pd.read_csv(data_path)
test_data = df_test['SQL_Statement'].tolist()  # Adjust column name as necessary
true_labels = df_test['Original'].apply(lambda x: 1 if x == 'YES' else 0).tolist()  # Adjust label processing as necessary

# Preprocess the test data
def preprocess_new_data(new_data):
    tokenized_data = tokenizer(new_data, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    return Dataset.from_dict({key: tokenized_data[key].detach().numpy() for key in tokenized_data})

test_dataset = preprocess_new_data(test_data)

# Predict using the trained model
predictions = trainer.predict(test_dataset)

# Extract logits and convert them to class predictions
predicted_classes = np.argmax(predictions.predictions, axis=1)

# Convert numeric predictions to labels
predicted_labels = ['YES' if label == 1 else 'NO' for label in predicted_classes]

# Evaluate the results
print("Predicted labels:", predicted_labels)
print(classification_report(true_labels, predicted_classes, target_names=['NO', 'YES']))



NameError: name 'tokenizer' is not defined